In [55]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

cap = cv2.VideoCapture( 'input.mov' )
isFirst = True  # （前回画像が無い）冒頭フレーム対応
while( cap.isOpened() ):
    ret, frame = cap.read()     # 動画からフレーム取得
    if ret:
        if isFirst: # i==0: #isFirst:
            prvs = frame[ :, :, 2 ] # 直前画像として保存 （緑色を使用）
            # 流れ速度を使った着色可視化用配列
            rgb = np.zeros_like( frame ); rgb[...,0] = 0
            fourcc = cv2.VideoWriter_fourcc(*'MJPG') # 処理結果保存用の動画コーデック
            # 処理結果を保存する動画ファイルを開く
            out = cv2.VideoWriter( 'result.avi', fourcc, 30, prvs.shape[ ::-1 ] )            
            # 速度場のXY座標群 (右辺座標はopenCV、左辺はmatplotlib)
            X, Y = np.meshgrid( np.arange( 0, frame.shape[1] ), 
                                           np.arange( frame.shape[0], 0, -1 ) )
        else:
            next = frame[ :, :, 2 ]
            flow = cv2.calcOpticalFlowFarneback(  # optical flow 算出 
                prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            prvs = next  # 処理したフレームを「前回画像」に格納
            rgb[..., 2] = cv2.normalize( flow[...,1], None, 0, 255, cv2.NORM_MINMAX )
            rgb[..., 1] = 255 - rgb[..., 2]            
            # 速度場 (右辺はopenCV座標、左辺はmatplotlib)
            dx = flow[...,0]; dy = flow[...,1] 
            # matplotlibで流れ場を描画
            fig = plt.figure( figsize = prvs.shape[::-1], dpi=1 )
            fig.patch.set_facecolor('black') 
            ax = fig.gca();  ax.axis('off')
            plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
            pitch = 15      # 流れ場を描く「間隔」
            plt.quiver( X[::pitch, ::pitch],    Y[ ::pitch, ::pitch ], 
                           dx[::pitch, ::pitch], -dy[::pitch, ::pitch],
                np.sqrt( dx[::pitch, ::pitch]**2 + dy[::pitch, ::pitch]**2 ),
                units="xy", scale=0.05, cmap="RdYlBu",
                linewidth=7 )
            fig.canvas.draw() # matplotlib レンダリング結果を画像に格納
            image = np.array( fig.canvas.renderer._renderer )
            plt.close(fig)
            # 動画として保存する
            rgb = cv2.addWeighted( rgb, 0.5, image[...,0:3], 0.5, 1.0)
            out.write( rgb )
        isFirst = False
    else:
        cap.release(); out.release() # 入力動画・出力動画ともに閉じる
        break